In [26]:
from collections import defaultdict, Counter
import numpy as np
from typing import List, Tuple, Type, Dict

with open("input.txt", "r") as f:
    lines = f.read().splitlines()

scanner_id = -1
scans = dict()

Coordinate = Tuple[int, int, int]

def dist(x, y):
    return set((abs(x[0]-y[0]), abs(x[1]-y[1]), abs(x[2]-y[2])))

class Scanner:
    def __init__(self, scanner_id :int):
        self.id :int= scanner_id
        self.beacons :List[Coordinate] = []
        self.distances :List[Tuple[int, int, Coordinate]] = []
        self.transformations : Dict[int, Tuple[np.array, np.matrix]] = dict()

    def add_beacon(self, coord : Coordinate):
        self.beacons.append(coord)
        if len(self.beacons) > 0:
            b1 = len(self.beacons)-1
            for i, b in enumerate(self.beacons[:-1]):
                self.distances.append((b1, i, dist(coord, b)))
                
    def beacons_for_scanner(self, scanner_id, beacons = None):
        beacons = self.beacons if beacons is None else beacons
        if scanner_id in self.transformations:
            d, a = self.transformations[scanner_id]
            beacons_coords = (a * np.matrix(beacons, dtype=int).transpose()).transpose() + d
            return list(map(tuple, beacons_coords.tolist()))
        else:
            raise RuntimeError(f"SCANNER {scanner_id} not found in transformations for scanner {self.id}")
    
def matching_distances(s1 :Scanner, s2: Scanner):
    matching = []
    for d1 in s1.distances:
        for d2 in s2.distances:
            if d1[2]==d2[2]:
                matching.append((set(d1[:2]), set(d2[:2])))
    return matching
   
scanner_id=0
for line in lines:
    if line.startswith("---"):
        scans[scanner_id] = Scanner(scanner_id)
        scanner = scans[scanner_id]
        scanner_id += 1
    elif line == "":
        continue
    else:
        scanner.add_beacon(tuple(map(int, line.split(","))))
    


In [35]:
def find_pivot_path(starting_scanner, pivot_scanner, scans, path = None):
    if path is None:
        path = []
    scanner = scans[starting_scanner]
    if (pivot_scanner in scanner.transformations.keys()):
        path.append(pivot_scan)
        return path
    else:
        for k in scanner.transformations.keys():
            if k not in path:
                path.append(k)
                r = find_pivot_path(k, pivot_scanner, scans, path)
                if r is not None:
                    return r
                path.remove(k)


for i1 in range(len(scans)):
    for i2 in range(len(scans)):
        if i2 != i1:
            s1 = scans[i1]
            s2 = scans[i2]
            d = matching_distances(s1, s2)
            matching_beacons = list()
            for i in range(len(s1.beacons)):
                m = list(map(lambda x: x[1], filter(lambda x: i in x[0], d)))
                if m:
                    matching_beacon = set.intersection(*m).pop()
                    matching_beacons.append((s1.beacons[i], s2.beacons[matching_beacon]))

            if len(matching_beacons)>=12:
                b1 = np.array(list(map(lambda x: x[0], matching_beacons)))
                b2 = np.array(list(map(lambda x: x[1], matching_beacons)))
                d1 = (b1[1, :] - b1[0, :]).flatten()
                d2 = (b2[1, :] - b2[0, :]).flatten()
                tm = []
                for i in range(3):
                    r = d2 / d1[i]
                    tm.append(np.where(np.logical_or(r==1, r==-1), r, 0))
                tm = np.matrix(tm, dtype=int)
                d = b1[0,:] - (tm * b2.transpose()).transpose()[0,:]
                scans[i2].transformations[i1] = (d, tm)

all_beacons = set()

c = Counter()
for i, s in scans.items():
    for k in list(s.transformations.keys()):
        c[k] += 1
pivot_scan = c.most_common()[0][0]
origins = {pivot_scan: (0,0,0)}
for i in range(len(scans)):
    if i==pivot_scan:
        all_beacons.update(scans[i].beacons)
    else:
        scanner = scans[i]
        if pivot_scan in scanner.transformations:
            bb = scanner.beacons_for_scanner(pivot_scan)
            all_beacons.update(bb)
        else:
            s = set(scanner.transformations.keys()).intersection(set(scans[pivot_scan].transformations.keys()))
            if len(s) > 0:
                common_scan = s.pop()
                tmp = scanner.beacons_for_scanner(common_scan)
                all_beacons.update(scans[common_scan].beacons_for_scanner(pivot_scan, tmp))
            else:
                found = False
                path = find_pivot_path(i, pivot_scan, scans)
                beacons = scanner.beacons
                origin = [(0,0,0)]
                sc = scanner
                for sc_id in path:
                    beacons = sc.beacons_for_scanner(sc_id, beacons)
                    origin = sc.beacons_for_scanner(sc_id, origin)
                    sc = scans[sc_id]
                origins[i] = origin[0] 
                all_beacons.update(beacons)

def manhattan_distance(a,b):
        return sum(abs(val1-val2) for val1, val2 in zip(a,b))
                   
answer_1 = len(all_beacons)

answer_2 = 0
for o1 in origins.values():
    for o2 in origins.values():
        d = manhattan_distance(o1, o2)
        answer_2 = max(answer_2, d)

        
print(f"answer_1: {answer_1}")
print(f"answer_2: {answer_2}")


answer_1: 403
answer_2: 10569
